In [7]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy


import good_model_working
from good_model_working.reload import deep_reload

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

In [9]:
# import the model data & sets
def load_data(filepath):        

# Load the pickled file
    with open(filepath, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)
    return data

model_data = load_data('/Users/peterambiel/Desktop/good_model/Model Input/all_input_objects.json')
model_sets = load_data('/Users/peterambiel/Desktop/good_model/Model Input/all_input_sets_sorted.json')

MemoryError: 

In [4]:
# create graph
graph = nx.node_link_graph(model_data, directed = True, multigraph = False)

In [6]:
# import good_model_working
deep_reload(good_model_working)

# run model 
problem = good_model_working.opt_model.Opt_Model(graph, model_sets)

Generating [{'data_type': 'generators', 'parameters': [{'plant_type': 'Biomass', 'generation': [{'fuel_type': 'Biomass', 'values': {'cost': 0.0, 'capacity': 100.0, 'group_id': 3.0}}]}]}, {'plant_type': 'Biomass', 'generation': [{'fuel_type': 'Biomass', 'values': {'cost': 0.0, 'capacity': 17.5, 'group_id': 4.0}}]}, {'plant_type': 'Coal Steam', 'generation': [{'fuel_type': 'Coal', 'values': {'cost': 57.582141455, 'capacity': 2920.0, 'group_id': 5.0}}]}, {'plant_type': 'Coal Steam', 'generation': [{'fuel_type': 'Coal', 'values': {'cost': 75.82390177356116, 'capacity': 10581.6, 'group_id': 6.0}}]}, {'plant_type': 'Combined Cycle', 'generation': [{'fuel_type': 'NaturalGas', 'values': {'cost': 39.56345955, 'capacity': 232.0, 'group_id': 7.0}}]}, {'plant_type': 'Combined Cycle', 'generation': [{'fuel_type': 'NaturalGas', 'values': {'cost': 24.328264287142854, 'capacity': 2292.0, 'group_id': 8.0}}]}, {'plant_type': 'Combined Cycle', 'generation': [{'fuel_type': 'NaturalGas', 'values': {'cost':

MemoryError: 

In [ ]:
# solve model
solver = pyomo.SolverFactory(solver_name)
solution = solver.solve(problem.model, tee=True)